In [1]:
import pandas as pd
import math
from datetime import date
import glob
import json
import numpy as np
import nbimporter
import sys
import numpy as np
import traceback
from os.path import exists

In [2]:
def load_data(file):
    daily = pd.read_csv(file)
    if 'Date' in daily.columns:
        daily['Date'] = pd.to_datetime(daily['Date'])
    else:
        daily['Date'] = pd.to_datetime(daily['Datetime'], utc=True).dt.tz_convert('US/Eastern')
    daily = daily.sort_values(by='Date', ascending=True)
    daily['Day'] = daily['Date'].dt.strftime('%m/%d/%Y')
    daily['Time'] = daily['Date'].dt.strftime('%H:%M')
    daily = daily.dropna().reset_index(drop=True)
    return daily

def load_yahoo_data(file):
    daily = pd.read_csv(file)
    if 'Date' in daily.columns:
        daily['Date'] = pd.to_datetime(daily['Date'])
    else:
        daily['Date'] = pd.to_datetime(daily['Datetime'], utc=True).dt.tz_convert('US/Eastern')
    daily = daily.sort_values(by='Date', ascending=True)
    daily['Day'] = daily['Date'].dt.strftime('%Y-%m-%d')
    daily['Time'] = daily['Date'].dt.strftime('%H:%M')
    daily = daily.dropna().reset_index(drop=True)
    return daily
def load_NKE_data():
    files = ["/Users/partha/Downloads/0-DTE/Stocks/NKE-20Year.csv",
            
            ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df

def load_stock_data(files):
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df

def load_stocks_data():
    files = ["/Users/partha/Downloads/0-DTE/stocks/GOOG-Dec-16.csv",
#              "/Users/partha/Downloads/0-DTE/indexes/NFLX-Dec-16.csv",
#              "/Users/partha/Downloads/0-DTE/indexes/GOOG-Dec-16.csv"
            ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
        df
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df
def load_all_5min_spx():
    files = [ 
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-13.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-16.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-12.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-9.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-8.csv",
            "/Users/partha/Downloads/0-DTE/indexes/SPX-DEC-6.csv",
             "/Users/partha/Downloads/0-DTE/indexes/SPX-DEC-7.csv",
            "/Users/partha/Downloads/0-DTE/indexes/spx-oct-28-snapshot.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-20.csv",
    "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-19.csv"]

    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df[df['Day'] > '08/31/2022']
    return df.drop_duplicates().sort_values(by='Date', ascending=True)
# df = load_all_5min_spx()
# df.head()

def load_5min_vix():
    files = [ 
        "/Users/partha/Downloads/0-DTE/indexes/VIX-Dec-9.csv",
        "/Users/partha/Downloads/0-DTE/indexes/VIX-Nov-09.csv",
        "/Users/partha/Downloads/0-DTE/indexes/VIX-Nov-09.csv"
    ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df[df['Day'] > '08/31/2022']
    return df.drop_duplicates().sort_values(by='Date', ascending=True)
    
def load_annual_data():
    files = ["/Users/partha/Downloads/0-DTE/indexes/SPX-20Year.csv",
            
            ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df



In [6]:
def build_seq(segment, split_size=10):
    
    rge = segment['High'].max() - segment['Low'].min()
    ilist = np.array_split(segment, split_size)
    amap = []
   
    for idf in ilist:
        try:
            idf = idf.reset_index(drop=True)
            s = int((idf[:1]['Open'].min()-segment['Low'].min())*split_size/rge)
            e = int((idf.iloc[-1]['Close'].min()-segment['Low'].min())*split_size/rge)
            if not amap:
                amap.append(s)
            else:    
                amap.append(e)
        except Exception as e:
            print ("Exception while building Sequence {}".format(e))
            pass    
    return amap


def predict(amap, seq_map, dates=None, times=None):
    lowest = sys.maxsize
    mystack = []
    match_seq = None
    for seq_key in seq_map.keys():
        seq = seq_map[seq_key]
        aseq = seq['Sequence']
        
        if aseq is None:
            continue
            
        if dates is not None:
            end_date = seq['Start Date']
            if end_date in dates: # and seq['Start Time'] == times[dates.index(end_date)]:
                continue
            else:
                pass

        try:
            dist = np.linalg.norm(np.array(aseq)-np.array(amap))
            if dist == 0:
                continue
                
            if dist < lowest:
                lowest = dist
                match_seq = str(aseq)
                mystack.append(seq)
            
        except Exception as e:
            #print (e)
            pass
    prediction = seq_map[match_seq].copy() if seq else {}
    try:
        prediction['Distance'] = lowest
        #prediction['stack'] = mystack
        pass
    except:
        pass
    return prediction


def test_a_segment(day, asegment, seqmap, split_size=20):
    asegment = asegment.reset_index(drop=True)
    #print ('Start date {} and End Date {}'.format(asegment.iloc[0]['Date'], asegment.iloc[-1]['Date']))
    #print ('lenth {}'.format(len(asegment)))
    if len(asegment) != split_size:
        print ("Input size and split size doesn't match {} / {}".format(len(asegment), split_size))
        return
    
    amap = build_seq(asegment, split_size)
    #print ('running prediction with {} and {} '.format(amap, seqmap))
    return predict(amap, seqmap)



In [47]:
def predict_stock(sym, horizon=9):
    df = load_yahoo_data('/Users/partha/experiments/candlestick-patterns/high-predictability-data/{}.csv'.format(sym))
    test_df = df[-20:]
    model = json.loads(open('/Users/partha/Downloads/0-DTE/Stocks/sp500-models-cleaned/{}.json'.format(sym)).read())
    match = test_a_segment(date, test_df, model, 20)
    pred_df = df[(df['Date'] >= match['End Date'])]
    pred = 1 if pred_df.iloc[horizon]['Close'] > pred_df.iloc[0]['Close'] else -1
    prediction = {'S': sym, 'D0': match['Start date'], 'D1': match['End Date'], 'Dist': match['Distance'], "P": pred}
    return prediction


In [48]:
predictions = []
for sym in pd.read_csv('/Users/partha/experiments/candlestick-patterns/High-Predictability-Symbols.csv')['Symbol']:
    predictions.append(predict_stock(sym))

pd.DataFrame(predictions).sort_values(by='Dist')

,S,D0,D1,Dist,P
6,EL,2007-09-18,2007-10-15,6.855655,1
1,ABMD,2014-10-17,2014-11-13,8.660254,1
3,TFX,2013-09-12,2013-10-09,9.055385,1
9,FE,2020-11-30,2020-12-28,9.433981,1
2,PG,2019-11-26,2019-12-24,9.695360,1
8,BEN,2018-04-11,2018-05-08,10.198039,1
0,YUM,2012-09-04,2012-10-01,12.165525,1
13,ETR,2016-03-14,2016-04-11,12.369317,1
11,CE,2005-08-08,2005-09-02,17.435596,1
10,GOOGL,2013-03-01,2013-03-28,18.681542,1


In [42]:
predict_stock('ORLY')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/partha/experiments/candlestick-patterns/high-predictability-data/ORLY.csv'